# MCP (Model Context Protocol) Tool Integration

This notebook demonstrates how to integrate MCP servers with LangChain agents using OCI Generative AI.

## What You'll Learn
- How to connect to MCP servers
- Manual tool calling with external services
- Agent execution with MCP tools

## Prerequisites
- `sandbox.yaml` configured with OCI credentials
- MCP servers running (weather server on port 8000)

## Documentation
- [MCP Specification](https://modelcontextprotocol.io/specification)
- [FastMCP](https://gofastmcp.com/getting-started/welcome)
- [LangChain MCP](https://docs.langchain.com/oss/python/langchain/mcp)
- [OCI Gen AI](https://docs.oracle.com/en-us/iaas/Content/generative-ai/home.htm)

## Step 1: Import Required Libraries

First, we need to import all the necessary libraries for our MCP integration.

**What this does:**
- Imports MCP client, LangChain components, and OCI helper
- Sets up the Python path to access our helper modules

**Why this matters:**
Proper imports ensure all components work together seamlessly.

In [ ]:
import sys, os
from dotenv import load_dotenv
from envyaml import EnvYAML
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.messages import HumanMessage

from langChain.oci_openai_helper import OCIOpenAIHelper

print("All libraries imported successfully!")

## Step 2: Load Configuration Files

Load the configuration files needed for OCI and environment setup.

**What this does:**
- Loads environment variables from `.env`
- Sets up the sandbox configuration file path
- Defines the LLM model to use

**Why this matters:**
Configuration files contain credentials and settings needed for OCI API access.

In [ ]:
# Load environment variables
load_dotenv()

# Configuration file paths and model settings
SANDBOX_CONFIG_FILE = "sandbox.yaml"
LLM_MODEL = "openai.gpt-4.1"

def load_config(config_path):
    """Load configuration from a YAML file."""
    try:
        with open(config_path, 'r') as f:
            return EnvYAML(config_path)
    except FileNotFoundError:
        print(f"Error: Configuration file '{config_path}' not found.")
        return None

print(f"Configuration loaded. Using model: {LLM_MODEL}")
print(f"Sandbox config file: {SANDBOX_CONFIG_FILE}")

## Step 3: Initialize OCI Client

Create the OCI Generative AI client that will power our LLM.

**What this does:**
- Loads the sandbox configuration
- Initializes the OCI OpenAI-compatible client
- Tests the connection

**Why this matters:**
This client handles all communication with OCI's AI models.

In [ ]:
# Load configuration
scfg = load_config(SANDBOX_CONFIG_FILE)

if scfg is None:
    print("ERROR: Could not load sandbox configuration!")
    print("Make sure sandbox.yaml exists and is properly configured.")
else:
    # Initialize OCI client
    llm_client = OCIOpenAIHelper.get_langchain_openai_client(
        model_name=LLM_MODEL,
        config=scfg
    )
    
    print("✅ OCI client initialized successfully!")
    print(f"Model: {LLM_MODEL}")

## Step 4: Define MCP Server Configuration

Set up the configuration for connecting to MCP servers.

**What this does:**
- Defines server endpoints and transport methods
- Weather server uses HTTP transport
- Bill server uses stdio (local process) transport

**Why this matters:**
MCP supports different transport protocols for various deployment scenarios.

In [ ]:
# MCP server configuration
MCP_SERVERS = {
    "weather": {
        "transport": "streamable_http",
        "url": "http://localhost:8000/mcp",
    },
    "bill_server": {
        "command": "python",
        "args": ["./langChain/function_calling/mcp/bill_mcp_server.py"],
        "transport": "stdio",
    },
}

print("MCP server configuration defined:")
for server_name, config in MCP_SERVERS.items():
    transport = config.get("transport")
    if transport == "streamable_http":
        print(f"- {server_name}: HTTP server at {config.get('url')}")
    elif transport == "stdio":
        print(f"- {server_name}: Local process via stdio")

## Step 5: Create MCP Client Function

Define a function to initialize the MCP client and connect to servers.

**What this does:**
- Creates MultiServerMCPClient instance
- Connects to all configured servers
- Retrieves available tools from servers

**Why this matters:**
This function handles the complex setup of MCP connections.

In [ ]:
async def create_mcp_client():
    """
    Create and initialize MCP client with all configured servers.
    
    Returns:
        client: Initialized MCP client
    """
    print("Creating MCP client...")
    
    client = MultiServerMCPClient(MCP_SERVERS)
    
    print("Connecting to MCP servers...")
    # The client will connect when we call get_tools()
    
    return client

# Create the MCP client
import asyncio
mcp_client = await create_mcp_client()
print("✅ MCP client created successfully!")

## Step 6: Discover Available Tools

Connect to MCP servers and discover what tools are available.

**What this does:**
- Actually connects to the servers
- Retrieves tool definitions from each server
- Lists all available tools with descriptions

**Why this matters:**
Before using tools, we need to know what they do and how to call them.

In [ ]:
# Get available tools from MCP servers
tools = await mcp_client.get_tools()

print(f"✅ Connected to MCP servers! Found {len(tools)} tools:")
print()

for i, tool in enumerate(tools, 1):
    print(f"{i}. {tool.name}")
    print(f"   Description: {tool.description}")
    print()

# Create a lookup dictionary for easy tool access
tools_by_name = {tool.name: tool for tool in tools}

print(f"Tool lookup dictionary created with {len(tools_by_name)} tools.")

## Step 7: Bind Tools to LLM

Connect the MCP tools to our LLM client so it can use them.

**What this does:**
- Binds tools to the LLM client
- Creates a "tooled" model that can call external functions

**Why this matters:**
The LLM needs to know about available tools to decide when to call them.

In [ ]:
# Bind tools to the LLM client
tooled_model = llm_client.bind_tools(tools)

print("✅ Tools bound to LLM client!")
print(f"Model now has access to {len(tools)} MCP tools.")
print()
print("The LLM can now:")
print("- Decide when to call tools based on user queries")
print("- Generate appropriate tool call parameters")
print("- Process tool results to generate final responses")

## Step 8: Test Single Tool Call

Test a simple query that should trigger just one tool.

**What this does:**
- Sends a simple query to the LLM
- Checks if the LLM wants to call any tools
- Executes the tool if requested

**Why this matters:**
Understanding single tool calls helps before moving to complex multi-tool scenarios.

In [ ]:
# Test with a simple weather query
test_query = "What's the weather like in San Francisco?"
messages = [HumanMessage(test_query)]

print(f"User query: {test_query}")
print("=" * 50)

# Get AI response
ai_message = tooled_model.invoke(messages)
print(f"AI Response: {ai_message.content}")

# Check for tool calls
tool_calls = getattr(ai_message, "tool_calls", None)
if tool_calls:
    print(f"\n🛠️  Tool calls detected: {len(tool_calls)}")
    
    # Add AI message to conversation
    messages.append(ai_message)
    
    # Execute the tool call
    for tool_call in tool_calls:
        tool_name = tool_call["name"]
        print(f"\nExecuting tool: {tool_name}")
        
        selected_tool = tools_by_name.get(tool_name)
        if selected_tool:
            tool_msg = await selected_tool.ainvoke(tool_call)
            print(f"Tool result: {tool_msg.content}")
            
            # Add tool result to conversation
            messages.append(tool_msg)
        else:
            print(f"❌ Tool '{tool_name}' not found!")
            
else:
    print("\n✅ No tool calls needed - direct response from LLM")

print("\n" + "=" * 50)
print("Single tool test completed!")

## Step 9: Manual Multi-Tool Conversation

Demonstrate a conversation that requires multiple tool calls.

**What this does:**
- Shows the complete conversation loop
- Handles multiple rounds of tool calling
- Processes complex queries requiring multiple tools

**Why this matters:**
Real-world applications often need multiple tools to answer complex questions.

In [ ]:
async def run_manual_conversation():
    """Run a manual conversation with multiple tool calls."""
    
    # Complex query requiring both weather and bill tools
    user_query = "What is my projected bill if the weather is 35 degrees and I have a gas oven? Also, are there any weather alerts for Colorado?"
    messages = [HumanMessage(user_query)]
    
    print(f"🎯 User query: {user_query}")
    print("=" * 70)
    
    conversation_round = 0
    
    while True:
        conversation_round += 1
        print(f"\n🔄 Round {conversation_round}:")
        
        # Get AI response
        ai_message = tooled_model.invoke(messages)
        print(f"🤖 AI: {ai_message.content}")
        
        # Check for tool calls
        tool_calls = getattr(ai_message, "tool_calls", None)
        if not tool_calls:
            print("\n✅ No more tool calls - conversation complete!")
            break
        
        print(f"\n🛠️  Tool calls in this round: {len(tool_calls)}")
        messages.append(ai_message)
        
        # Execute tools
        for i, tool_call in enumerate(tool_calls, 1):
            tool_name = tool_call["name"]
            print(f"\n  {i}. Executing: {tool_name}")
            
            selected_tool = tools_by_name.get(tool_name)
            if selected_tool:
                tool_msg = await selected_tool.ainvoke(tool_call)
                print(f"     Result: {tool_msg.content[:100]}..." if len(tool_msg.content) > 100 else f"     Result: {tool_msg.content}")
                messages.append(tool_msg)
            else:
                print(f"     ❌ Tool '{tool_name}' not found!")
    
    return messages

# Run the conversation
conversation_history = await run_manual_conversation()
print(f"\n📊 Conversation completed with {len(conversation_history)} messages!")

## Step 10: Agent-Style Execution

Show how the manual approach can be wrapped in an agent-like function.

**What this does:**
- Creates a reusable function for agent execution
- Demonstrates the pattern used in langchain_host.py
- Shows how to handle different types of queries

**Why this matters:**
This bridges manual tool calling with automated agent behavior.

In [13]:
async def mcp_agent(query):
    """
    Agent-style execution with MCP tools.
    Similar to the pattern in langchain_host.py
    """
    print(f"🤖 Agent processing: {query}")
    print("=" * 60)
    
    messages = [HumanMessage(query)]
    total_tools_called = 0
    
    while True:
        # Get AI response
        ai_message = tooled_model.invoke(messages)
        print(f"\n🤖 AI: {ai_message.content}")
        
        # Check for tool calls
        tool_calls = getattr(ai_message, "tool_calls", None)
        if not tool_calls:
            break
        
        messages.append(ai_message)
        
        # Execute tools
        for tool_call in tool_calls:
            tool_name = tool_call["name"]
            selected_tool = tools_by_name.get(tool_name)
            
            if selected_tool:
                tool_msg = await selected_tool.ainvoke(tool_call)
                messages.append(tool_msg)
                total_tools_called += 1
                print(f"🛠️  Called {tool_name}")
    
    print(f"\n✅ Agent completed! Used {total_tools_called} tools.")
    return messages

# Test the agent with different queries
test_queries = [
    "What's the weather in Denver?",
    "Calculate my bill for 50 degrees with electric oven",
    "Weather alerts for California and bill projection for 40 degrees gas oven"
]

for query in test_queries:
    await mcp_agent(query)
    print("\n" + "-" * 60)


🤖 AI: Here are the latest weather alerts for California and your gas bill projection for 40°F with a gas oven:

### California Weather Alerts
- Multiple Flood Advisories and Flash Flood Warnings are in effect for areas including Los Angeles, Ventura, Orange, Santa Barbara, San Bernardino, San Diego, and surrounding counties.
- Impacts include minor to severe flooding, rock and mud slides, dangerous surf conditions, possible road closures, and life-threatening conditions near burn scars.
- Winter Weather Advisory: Up to 11 inches of snow above 7000 feet in parts of the Sierra Nevada.
- High Surf and Beach Hazards: Dangerous rip currents and waves up to 14 feet along coastal areas.
- Severe flood watches continue throughout the day for many parts of the state due to heavy rain from atmospheric rivers.
- See full alert details and stay updated: [National Weather Service Flood Safety](http://www.weather.gov/safety/flood)

**Instructions:** Avoid flooded roadways, heed evacuation orders if

## Exercises

### Exercise 1: Single Tool Queries
Try queries that only use one tool:
- "What's the weather in New York?"
- "What's my bill projection for 60 degrees with electric oven?"

### Exercise 2: Multi-Tool Queries
Create queries requiring both tools:
- "If it's 45 degrees and I have gas heating, what's my bill and any alerts for Texas?"

### Exercise 3: Error Handling
Test what happens when servers aren't running:
- Stop the weather server and try a weather query
- Observe how the system handles connection failures

### Exercise 4: Model Comparison
Change the LLM_MODEL and compare results:
- Try 'openai.gpt-4o' or 'xai.grok-4'
- See how different models handle tool calling

### Exercise 5: Custom Queries
Create your own test queries and observe the tool calling behavior.